# Setup Guide

### Prerequisites
1. Create an account on [Ragas app](https://dev.app.ragas.io/)
2. Obtain your [App TOKEN](https://dev.app.ragas.io/dashboard/settings/app-tokens) from the dashboard
3. Have or create an [OpenAI API KEY](https://openai.com/)

### Setting Up Observability

1. Navigate to the linkedin_ai directory:
   ```bash
   cd linkedin_ai
   ```

2. Host mlflow UI and point to the URI
    ```bash
    mlflow ui --port 8080 --backend-store-uri file:///<your-file-path>/linkedin_ai/mlruns    
    ```

In [ ]:
import os
os.environ["RAGAS_APP_TOKEN"] = ""
os.environ["RAGAS_API_BASE_URL"] = "https://api.dev.app.ragas.io"
os.environ["OPENAI_API_KEY"] =  ""

In [ ]:
import os
import mlflow

current_dir = os.getcwd()
tracking_uri = os.path.join(current_dir, "mlruns")
tracking_uri = f"file:///{tracking_uri.lstrip('/')}"

os.environ["MLFLOW_HOST"] = "http://127.0.0.1:8080"
os.environ["MLFLOW_TRACKING_URI"] = tracking_uri
mlflow.set_tracking_uri(tracking_uri)


In [ ]:
# mlflow uses this to log your traces
mlflow.create_experiment(
    "my_hackathon_experiment",

)


<Experiment: artifact_location='file:///Users/shahules/Myprojects/linkedin_ai/mlruns/481371020783987734', creation_time=1744920240094, experiment_id='481371020783987734', last_update_time=1744920240094, lifecycle_stage='active', name='my_hackathon_experiment', tags={}>

In [3]:
mlflow.set_experiment(
    "my_hackathon_experiment",

)

<Experiment: artifact_location='file:///Users/shahules/Myprojects/linkedin_ai/mlruns/481371020783987734', creation_time=1744920240094, experiment_id='481371020783987734', last_update_time=1744920240094, lifecycle_stage='active', name='my_hackathon_experiment', tags={}>

### Load your linkedin_ai endpoint

In [12]:
from linkedin_ai import LinkedinAI
my_ai = await LinkedinAI.from_bm25('data/posts.json')


Loaded 437 LinkedIn posts
BM25 index initialized


In [29]:
await my_ai.ask("what is your take on opensource models")

"Meta's open-source approach, particularly with models like LLAMA, is something I support. Open-sourcing models can foster innovation and collaboration within the AI community. It allows researchers and developers to build upon existing work, accelerating progress and enabling a broader range of applications. This approach aligns with the vision of creating AI systems that can understand and interact with the physical world, ultimately benefiting society as a whole."

### Read hackathon test dataset from Ragas App

In [4]:
from ragas_experimental import BaseModel

class TestDataset(BaseModel):
    question: str
    citations: str
    grading_notes: str

In [5]:
from ragas_experimental import Project

p = Project.get(name="yann-lecun-wisdom")

In [6]:
test_dataset = p.get_dataset(dataset_name="test-yann-lecun", model=TestDataset)
test_dataset.load()

In [7]:
test_dataset[0]

TestDataset(question='Where did you do your PhD and what was your field of study?', citations='["https://www.linkedin.com/in/yann-lecun"]', grading_notes='- Specifies Pierre and Marie Curie University.\n- Field of study should be computer science.\n- Bonus if year range (1983–1987) is included.')

### Create LLM as judge

In [8]:
# de
from ragas_experimental.llm import ragas_llm
from ragas_experimental.metric import DiscreteMetric
from openai import AsyncOpenAI

llm = ragas_llm(provider="openai",model="gpt-4o",client=AsyncOpenAI())

my_metric = DiscreteMetric(
    llm=llm,
    name='correctness',
    prompt="Given the Question: {query} \n Evaluate if given answer {response} \n based on the Grading notes\n: {grading_notes}.",
    values=["pass","fail"],
)



# test LLM as judge
result = my_metric.score(query="what is your response", response="this is my response",grading_notes="- response should not contains word response")
result

'fail'

### Run your first experiment

In [9]:
from ragas_experimental.tracing.mlflow import sync_trace

In [10]:
import typing as t
import ragas_experimental.typing as rt

class ExperimentModel(TestDataset):
    response: str
    score: t.Annotated[t.Literal["pass","fail"], rt.Select(colors=["green","red"])]
    score_reason: str
    trace_url:  t.Annotated[str, rt.Url()]
    

@p.mlflow_experiment(ExperimentModel)
async def experiment_func(item: TestDataset):
    response = await my_ai.ask(item.question)
    trace = await sync_trace()
    score = await my_metric.ascore(query=item.question, response=response, grading_notes=item.grading_notes)
    return ExperimentModel(question=item.question, citations=item.citations, grading_notes=item.grading_notes, response=response, score=score.result, score_reason=score.reason,trace_url = trace.get_url())

In [18]:
await experiment_func.run_async(test_dataset)

Running experiment: 100%|██████████| 100/100 [00:36<00:00,  2.73it/s]


Staging changes to tracked files
Changes committed with hash: 8d1c8d8e
Created branch: ragas/stoic_adleman


Experiment(name=stoic_adleman, model=ExperimentModel)

Go to Experiments tab in Ragas app and view the results

### Run your next experiment
- Make a change in your app code/ endpoint
- Here I am changing how many chunks I retrieve


In [ ]:
my_ai = await LinkedinAI.from_bm25('data/posts.json',top_k=5)


- Run another experiment

In [14]:
await experiment_func.run_async(test_dataset)

Running experiment: 100%|██████████| 100/100 [00:44<00:00,  2.25it/s]

Staging changes to tracked files
Changes committed with hash: a0692b82
Created branch: ragas/elegant_rivest


Experiment(name=elegant_rivest, model=ExperimentModel)

Go to Experiments tab in Ragas app and view the results

### Compare Experiments
- Go to Experiments tab in Ragas app and view different experiments
- Compare the experiments and see what works best for you

## Advanced
### LLM as judge giving incorrect results?
- Train it. 

1. Review and change incorrect results in Experiments tab
2. optimize llm as judge from library

In [ ]:
from ragas_experimental.embedding import ragas_embedding

from openai import OpenAI
embedding = ragas_embedding(provider='openai',client=OpenAI(),model="text-embedding-3-small")
my_metric.train(p,experiment_names=['<your-experiment-name-here>'],embedding_model=embedding,model=ExperimentModel)